### Sequential Model
A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.  
A Sequential model is not appropriate when:

1. Your model has multiple inputs or multiple outputs
2. Any of your layers has multiple inputs or multiple outputs
3. You need to do layer sharing
4. You want non-linear topology (e.g. a residual connection, a multi-branch model)

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pprint import pprint
np.random.seed(100)
tf.random.set_seed(100)

In [ ]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
train_df

In [ ]:
labelencoder = LabelEncoder()
train_df['thal'] = labelencoder.fit_transform(train_df['thal'])
test_df['thal'] = labelencoder.transform(test_df['thal'])
train_df

In [ ]:
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=133, shuffle= True)
train_labels = train_data.pop('target')
val_labels = val_data.pop('target')
test_labels = test_df.pop('target')

In [ ]:
# train_data.loc[:,['age','trestbps']].describe()
train_data

In [ ]:
def create_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(units = 6, 
                          activation='relu',
                          use_bias = True,
                          kernel_initializer = 'Ones',
                          bias_initializer='zeros',
                          kernel_regularizer=None, 
                          bias_regularizer=None,
                          activity_regularizer=None, 
                          kernel_constraint=None, 
                          bias_constraint=None,
                          input_shape=(13,), 
                          name = 'd1'),
    tf.keras.layers.Dense(units = 2,activation = 'softmax', name = 'out')
    ])
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def create_model1():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units = 6, 
                                    activation='relu', 
                                    name = 'd1', input_shape = (13,)))
    model.add(tf.keras.layers.Dense(units = 2, activation='softmax', name = 'out'))
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


model = create_model1()
tf.keras.utils.plot_model(model, 'my_first_model.png', show_shapes=True)
model.summary()

In [ ]:
x = tf.ones((240, 13))
y = model(x)
y

In [ ]:
#listing out all layers
model.layers

In [ ]:
layer1 = model.get_layer(name = 'd1')
sns.heatmap(layer1.weights[0].numpy())

In [ ]:
import json
jstring = model.to_json()

obj = json.loads(jstring)
obj

In [ ]:
print(model.to_yaml())

In [ ]:
model.save_weights("initials.h5")

In [ ]:
# model.load_weights("initials.h5")
history = model.fit(
    x = train_data, 
    y = train_labels,
    batch_size = 192,
    epochs = 5,
    verbose = 1,
    callbacks = None,
    validation_data = (val_data, val_labels),
    shuffle = True,
    class_weight = {0:0.5, 1:2}
)

In [ ]:
pprint(history.history)

In [ ]:
test_loss, test_acc = model.evaluate(
    x = val_data, 
    y = val_labels,
    batch_size = 32,
    verbose = 1
)

In [ ]:
predictions_probabilities = model.predict(test_df)
predictions_probabilities

In [ ]:
predictions = tf.math.argmax(predictions_probabilities, 1).numpy()
predictions

In [ ]:
np.array(test_labels)

In [ ]:
cm = tf.math.confusion_matrix(
    test_labels, predictions,
    num_classes=2).numpy()
cm


In [ ]:
model.save('mymodel.h5')

In [ ]:
del model

In [ ]:
model = tf.keras.models.load_model('mymodel.h5')

In [ ]:
model.reset_metrics()

In [ ]:
test_loss, test_acc = model.evaluate(
    x = val_data, 
    y = val_labels,
    batch_size = 32,
    verbose = 1
)

In [ ]:
predictions_probabilities1 = model.predict(test_df)
predictions_probabilities1

In [ ]:
predictions_probabilities1